In [3]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [5]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16669 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up software-properties-common (0.96.24.17) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing triggers for dbus (1.10.22-1ubuntu1) ...
gpg: keybox '/tmp/tmpmt_39nn4/pubring.gpg' created
gpg: /tmp/tmpmt_39nn4/trustdb.g

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [7]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

In [32]:
!ls drive/club_data/

POST  PRE


In [0]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
from matplotlib import pyplot as plt
import shutil
import os

In [0]:
BASE_DIR = '/drive/club_data/'

In [0]:
# Function to split all data directory into training and validation

def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        category_name = os.path.basename(subdir)

        # Don't create a subdirectory for the root directory
        print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        for file in files:
            input_file = os.path.join(subdir, file)
            if np.random.rand(1) < testing_data_pct:
                shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                num_testing_files += 1
            else:
                shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                num_training_files += 1

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [34]:
! ls drive/CSE_project

club_data  train  valid


In [39]:
all_data_dir = 'drive/CSE_project/club_data/PRE'

training_data_dir = 'drive/CSE_project/pre_train'

testing_data_dir ='drive/CSE_project/pre_valid'

split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir,  0.2)

Successfully cleaned directory drive/CSE_project/pre_valid
Successfully cleaned directory drive/CSE_project/pre_train
PRE vs PRE
BAD vs PRE
GOOD vs PRE
Processed 46 training files.
Processed 14 testing files.


In [40]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        training_data_dir,  # this is the target directory
        target_size=(512, 512),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        testing_data_dir,
        target_size=(512, 512),
        batch_size=14,
        class_mode='binary')

Found 46 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [0]:
# Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(512, 512, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [43]:
model.fit_generator(
        train_generator,
        steps_per_epoch=46 // batch_size,
        epochs=300,
        validation_data=validation_generator,
        validation_steps=1)

model.save_weights('first_try.h5') 

Epoch 1/300
2/2 [==============================] - 20s 10s/step - loss: 2.6216 - acc: 0.6308 - val_loss: 4.5550 - val_acc: 0.7143
Epoch 2/300
2/2 [==============================] - 14s 7s/step - loss: 3.7796 - acc: 0.5938 - val_loss: 0.6163 - val_acc: 0.7143
Epoch 3/300
2/2 [==============================] - 11s 6s/step - loss: 1.2753 - acc: 0.5579 - val_loss: 1.0431 - val_acc: 0.2857
Epoch 4/300
2/2 [==============================] - 11s 6s/step - loss: 0.9599 - acc: 0.3722 - val_loss: 0.6493 - val_acc: 0.7143
Epoch 5/300
2/2 [==============================] - 12s 6s/step - loss: 0.6080 - acc: 0.6250 - val_loss: 0.6677 - val_acc: 0.7143
Epoch 6/300
2/2 [==============================] - 11s 6s/step - loss: 0.7026 - acc: 0.5988 - val_loss: 0.7409 - val_acc: 0.2857
Epoch 7/300
2/2 [==============================] - 11s 6s/step - loss: 0.8078 - acc: 0.3663 - val_loss: 0.7311 - val_acc: 0.2857
Epoch 8/300
2/2 [==============================] - 12s 6s/step - loss: 0.7109 - acc: 0.5000 - va

2/2 [==============================] - 11s 5s/step - loss: 0.6807 - acc: 0.6018 - val_loss: 0.6540 - val_acc: 0.7143
Epoch 24/300
2/2 [==============================] - 11s 5s/step - loss: 0.6600 - acc: 0.6976 - val_loss: 0.6529 - val_acc: 0.7143
Epoch 25/300
2/2 [==============================] - 11s 6s/step - loss: 0.6748 - acc: 0.6337 - val_loss: 0.6491 - val_acc: 0.7143
Epoch 26/300
2/2 [==============================] - 11s 6s/step - loss: 0.6497 - acc: 0.6308 - val_loss: 0.6314 - val_acc: 0.7143
Epoch 27/300
2/2 [==============================] - 12s 6s/step - loss: 0.6477 - acc: 0.6875 - val_loss: 0.6156 - val_acc: 0.7143
Epoch 28/300
2/2 [==============================] - 11s 5s/step - loss: 0.6624 - acc: 0.7036 - val_loss: 0.6037 - val_acc: 0.7143
Epoch 29/300
2/2 [==============================] - 12s 6s/step - loss: 0.6930 - acc: 0.5625 - val_loss: 0.6050 - val_acc: 0.7143
Epoch 30/300
2/2 [==============================] - 11s 5s/step - loss: 0.5935 - acc: 0.7006 - val_loss

2/2 [==============================] - 11s 6s/step - loss: 0.6898 - acc: 0.6077 - val_loss: 0.6462 - val_acc: 0.7143
Epoch 46/300
2/2 [==============================] - 11s 6s/step - loss: 0.6935 - acc: 0.6337 - val_loss: 0.6521 - val_acc: 0.7143
Epoch 47/300
2/2 [==============================] - 11s 6s/step - loss: 0.7011 - acc: 0.5958 - val_loss: 0.6595 - val_acc: 0.7143
Epoch 48/300
2/2 [==============================] - 12s 6s/step - loss: 0.6133 - acc: 0.7188 - val_loss: 0.6611 - val_acc: 0.7143
Epoch 49/300
2/2 [==============================] - 11s 5s/step - loss: 0.6079 - acc: 0.7325 - val_loss: 0.6550 - val_acc: 0.7143
Epoch 50/300
2/2 [==============================] - 11s 5s/step - loss: 0.7055 - acc: 0.5698 - val_loss: 0.6604 - val_acc: 0.7143
Epoch 51/300
2/2 [==============================] - 11s 6s/step - loss: 0.6617 - acc: 0.6562 - val_loss: 0.6585 - val_acc: 0.7143
Epoch 52/300
2/2 [==============================] - 11s 5s/step - loss: 0.6267 - acc: 0.7296 - val_loss

KeyboardInterrupt: ignored